In [308]:
import pandas as pd
from qiskit_machine_learning.connectors import TorchConnector
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap
from qiskit_machine_learning.neural_networks import SamplerQNN, EstimatorQNN
import torch

data = pd.read_csv("./data/dsc_fc_summed_spectra_2020_v01.csv", \
delimiter = ',', parse_dates=[0], na_values='0', \
infer_datetime_format=True, \
header = None)

label1 = pd.read_csv("./data/2020Q1_DGD.csv", \
delimiter = ',', parse_dates=[0], \
infer_datetime_format=True, \
header = None)

label2 = pd.read_csv("./data/2020Q2_DGD.csv", \
delimiter = ',', parse_dates=[0], \
infer_datetime_format=True, \
header = None)

label3 = pd.read_csv("./data/2020Q3_DGD.csv", \
delimiter = ',', parse_dates=[0], \
infer_datetime_format=True, \
header = None)

label4 = pd.read_csv("./data/2020Q4_DGD.csv", \
delimiter = ',', parse_dates=[0], \
infer_datetime_format=True, \
header = None)

/var/folders/tx/xsb7n6wx3m3_hgtjstktffs80000gn/T/ipykernel_90254/2591706683.py:7: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  data = pd.read_csv("./data/dsc_fc_summed_spectra_2020_v01.csv", \
/var/folders/tx/xsb7n6wx3m3_hgtjstktffs80000gn/T/ipykernel_90254/2591706683.py:12: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  label1 = pd.read_csv("./data/2020Q1_DGD.csv", \
/var/folders/tx/xsb7n6wx3m3_hgtjstktffs80000gn/T/ipykernel_90254/2591706683.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateu

In [309]:
labels = pd.concat([label1, label2, label3, label4], ignore_index=True, sort=False)

In [310]:
data.fillna('0', inplace=True)

In [311]:
data.iloc[:, 1:49] = data.iloc[:, 1:49].apply(pd.to_numeric, errors='coerce')

grouped_by_day = data.groupby(data.iloc[:, 0].dt.date)
day_dataframes = [group for _, group in grouped_by_day]

filtered_data0003 = []

filtered_data0306 = []

filtered_data0609 = []

filtered_data0912 = []

filtered_data1215 = []

filtered_data1518 = []

filtered_data1821 = []

filtered_data2123 = []

counter = 0
for day_df in day_dataframes:
    filtered_data0003.append(day_df[day_df.iloc[:, 0].dt.hour.between(0, 2)])

    filtered_data0306.append(day_df[day_df.iloc[:, 0].dt.hour.between(3, 5)])

    filtered_data0609.append(day_df[day_df.iloc[:, 0].dt.hour.between(6, 8)])

    filtered_data0912.append(day_df[day_df.iloc[:, 0].dt.hour.between(9, 11)])

    filtered_data1215.append(day_df[day_df.iloc[:, 0].dt.hour.between(12, 14)])

    filtered_data1518.append(day_df[day_df.iloc[:, 0].dt.hour.between(15, 17)])

    filtered_data1821.append(day_df[day_df.iloc[:, 0].dt.hour.between(18, 20)])

    filtered_data2123.append(day_df[day_df.iloc[:, 0].dt.hour.between(21, 23)])
    counter += 1


In [312]:
day_dataframes[ -180:-1]

[                        0        1        2        3    4    5    6    7   \
 188640 2020-07-05 00:00:00 -6.65234  6.65881  2.10804  0.0  0.0  0.0  0.0   
 188641 2020-07-05 00:01:00 -7.10133  6.46347  3.05658  0.0  0.0  0.0  0.0   
 188642 2020-07-05 00:02:00 -7.59377  5.87116     3.21  0.0  0.0  0.0  0.0   
 188643 2020-07-05 00:03:00 -7.88376  5.55128  2.93704  0.0  0.0  0.0  0.0   
 188644 2020-07-05 00:04:00 -7.76079   5.4959  3.14622  0.0  0.0  0.0  0.0   
 ...                    ...      ...      ...      ...  ...  ...  ...  ...   
 190075 2020-07-05 23:55:00 -3.58551  1.91148  3.10241  0.0  0.0  0.0  0.0   
 190076 2020-07-05 23:56:00  -3.5093  2.18199  3.07733  0.0  0.0  0.0  0.0   
 190077 2020-07-05 23:57:00 -3.49265  2.18834  3.02237  0.0  0.0  0.0  0.0   
 190078 2020-07-05 23:58:00 -3.61224  2.16792  2.91679  0.0  0.0  0.0  0.0   
 190079 2020-07-05 23:59:00 -3.59361  2.38305  2.79861  0.0  0.0  0.0  0.0   
 
          8    9   ...   44   45   46   47   48 49 50 51 52 53

In [313]:
mean_val1 = []
mean_val2 = []
mean_val3 = []
mean_val4 = []
mean_val5 = []
mean_val6 = []
mean_val7 = []
mean_val8 = []

for i in range(311):
    mean_val1.append(filtered_data0003[i].iloc[:, 1:49].mean(axis=0).values)
    mean_val2.append(filtered_data0306[i].iloc[:, 1:49].mean(axis=0).values)
    mean_val3.append(filtered_data0609[i].iloc[:, 1:49].mean(axis=0).values)
    mean_val4.append(filtered_data0912[i].iloc[:, 1:49].mean(axis=0).values)
    mean_val5.append(filtered_data1215[i].iloc[:, 1:49].mean(axis=0).values)
    mean_val6.append(filtered_data1518[i].iloc[:, 1:49].mean(axis=0).values)
    mean_val7.append(filtered_data1821[i].iloc[:, 1:49].mean(axis=0).values)
    mean_val8.append(filtered_data2123[i].iloc[:, 1:49].mean(axis=0).values)


In [314]:
mean_val2[0]

array([3.8107355000000003, -1.9185921111111097, 0.3562674956666667, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0], dtype=object)

In [315]:
sum = []

for i in range(311):
    sum.append(pd.DataFrame([mean_val1[i], mean_val2[i], mean_val3[i], mean_val4[i], mean_val5[i], mean_val6[i], mean_val7[i], mean_val8[i]], columns=data.columns[1:49]).astype(str))

In [316]:
sum[0]

,1,2,3,4,5,6,7,8,9,10,...,39,40,41,42,43,44,45,46,47,48
0,3.743139611111113,-1.7453320500000005,-0.09027723151666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3.8107355000000003,-1.9185921111111097,0.3562674956666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.126327666666667,-1.048211594444445,0.5624192355000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2.872449472222221,-1.9870318388888886,-0.21568646555555546,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.815778777777778,-1.6170664666666659,0.11151597866666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,3.247966055555556,-0.6738210188888891,0.42698228666666665,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,3.098512888888886,-1.2162829133333335,0.26739942772222225,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,3.505577277777775,-1.0348612511111113,-0.06844686555555554,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [317]:
sum[-1]

,1,2,3,4,5,6,7,8,9,10,...,39,40,41,42,43,44,45,46,47,48
0,-0.6909120984,-2.0459063666666655,-0.2624655994444446,58.952142777777745,0.3521973277777777,2.33432421111111,21.177793722222226,84.70042611111117,54.987448333333326,30.359590944444456,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.1102529670555556,-1.9611446000000003,-0.0011972236111109267,71.03827611111112,0.42749231666666626,4.455874499999993,13.015458888888887,86.55190888888892,66.81519611111109,33.36408949999999,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.7427560659444444,0.08757565499999997,-1.2173098661111112,35.184371111111105,0.10695035555555543,3.4174273944444447,3.6706029833333327,31.06412166666668,35.024834999999996,21.30574833333334,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.12032895555555567,-0.8425215116666662,0.5978537276111108,35.284063333333336,0.2593102666666664,33.49888388888888,35.55578055555558,55.66307555555555,46.95401555555554,55.964914444444474,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.8891119038888889,-0.9926265402222223,0.222668428888889,38.068741111111116,0.2696034944444446,33.476967777777766,36.37692222222221,61.32444333333333,50.593615555555544,60.70308444444447,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,-0.8575922298888888,-1.5208163427777779,-0.17299531777777796,43.21426055555554,0.29991106111111165,30.768869444444437,33.71664499999999,68.41786,55.98023944444448,64.3466966666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.6829002322222218,-0.9465150132777777,0.05532093338888882,36.97416166666668,0.25838764444444434,42.01192222222222,38.32426500000002,54.73010444444447,52.66074777777777,80.3296872222222,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1.181123496183334,-0.16441391111111123,0.48781959000000025,18.410726666666665,0.1197250999999998,22.091523888888894,20.87331611111112,28.510348333333333,26.174248333333335,40.56894166666665,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [318]:
sorted_df = pd.concat(sum).reset_index(drop=True)

sorted_df

,1,2,3,4,5,6,7,8,9,10,...,39,40,41,42,43,44,45,46,47,48
0,3.743139611111113,-1.7453320500000005,-0.09027723151666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3.8107355000000003,-1.9185921111111097,0.3562674956666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.126327666666667,-1.048211594444445,0.5624192355000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2.872449472222221,-1.9870318388888886,-0.21568646555555546,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.815778777777778,-1.6170664666666659,0.11151597866666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2483,-0.12032895555555567,-0.8425215116666662,0.5978537276111108,35.284063333333336,0.2593102666666664,33.49888388888888,35.55578055555558,55.66307555555555,46.95401555555554,55.964914444444474,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2484,-0.8891119038888889,-0.9926265402222223,0.222668428888889,38.068741111111116,0.2696034944444446,33.476967777777766,36.37692222222221,61.32444333333333,50.593615555555544,60.70308444444447,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2485,-0.8575922298888888,-1.5208163427777779,-0.17299531777777796,43.21426055555554,0.29991106111111165,30.768869444444437,33.71664499999999,68.41786,55.98023944444448,64.3466966666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2486,0.6829002322222218,-0.9465150132777777,0.05532093338888882,36.97416166666668,0.25838764444444434,42.01192222222222,38.32426500000002,54.73010444444447,52.66074777777777,80.3296872222222,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [319]:
labels = labels[55: 366]

labels.reset_index(drop=True, inplace=True)


In [320]:
labels

,0,1,2,3,4,5,6,7
0,2,0,1,0,1,0,0,0
1,1,1,1,0,1,1,2,1
2,2,1,1,1,1,0,1,1
3,2,2,1,1,1,1,1,2
4,2,2,2,2,2,3,2,4
...,...,...,...,...,...,...,...,...
306,2,2,1,2,2,1,1,2
307,2,2,2,2,2,1,1,2
308,3,2,1,1,1,2,1,3
309,3,3,2,2,2,1,1,3


In [321]:
labels = labels.stack()

labels =pd.DataFrame(labels, columns=["Stacked_Column"])



In [322]:
labels.reset_index(drop=True, inplace=True)

labels

,Stacked_Column
0,2
1,0
2,1
3,0
4,1
...,...
2483,1
2484,0
2485,0
2486,1


In [323]:
X = sorted_df
Y = labels

In [324]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from qiskit_machine_learning.connectors import TorchConnector

import torch

In [325]:
X = X.to_numpy(dtype=np.float32)
Y = Y.to_numpy(dtype=np.float32)  # Cast Y to np.float32 as well

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, shuffle=False)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

batch_size = 32  # You can adjust this batch size as needed
train_dataset = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(Y_train))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
val_dataset = TensorDataset(torch.from_numpy(X_val), torch.from_numpy(Y_val))
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [326]:
X_val.shape

(498, 48)

In [327]:
num_epochs = 50  # Adjust as needed
learning_rate = 0.001  # Adjust as needed

# Define your optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

In [328]:
def create_qnn():
    feature_map = ZZFeatureMap(2)
    ansatz = RealAmplitudes(2, reps=1)
    qc = QuantumCircuit(2)
    qc.compose(feature_map, inplace=True)
    qc.compose(ansatz, inplace=True)

    # REMEMBER TO SET input_gradients=True FOR ENABLING HYBRID GRADIENT BACKPROP
    qnn = EstimatorQNN(
        circuit=qc,
        input_params=feature_map.parameters,
        weight_params=ansatz.parameters,
        input_gradients=True,
    )
    return qnn


qnn4 = create_qnn()

In [329]:
from qiskit_machine_learning.connectors import TorchConnector
import torch
import torch.nn as nn
from torch import Tensor
from torch.nn import Linear, CrossEntropyLoss, MSELoss
from torch.optim import LBFGS

from qiskit import QuantumCircuit
from qiskit.utils import algorithm_globals
from qiskit.circuit import Parameter
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap
from qiskit_machine_learning.neural_networks import SamplerQNN, EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector

class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.lstm = nn.LSTM(input_size=48, hidden_size=64, batch_first=True, num_layers=1, bidirectional=False)
        self.dropout = nn.Dropout(0.7)
        self.dense1 = nn.Linear(64, 2)
        self.qnn4 = TorchConnector(qnn4)  # Assuming 'qnn4' is your custom layer
        self.dense2 = nn.Linear(2, 1)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.dropout(x)
        x = self.dense1(x[:, -1, :])  # Taking the last output from the LSTM
        x = self.qnn4(x)
        x = self.dense2(x)
        return x

# Instantiate the model
model = CustomModel()

# Print the model architecture
print(model)

# Define your optimizer and loss function
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.MSELoss()

# Print a summary of the model (number of parameters, layer details, etc.)
print(model)

CustomModel(
  (lstm): LSTM(48, 64, batch_first=True)
  (dropout): Dropout(p=0.7, inplace=False)
  (dense1): Linear(in_features=64, out_features=2, bias=True)
  (qnn4): TorchConnector()
  (dense2): Linear(in_features=2, out_features=1, bias=True)
)
CustomModel(
  (lstm): LSTM(48, 64, batch_first=True)
  (dropout): Dropout(p=0.7, inplace=False)
  (dense1): Linear(in_features=64, out_features=2, bias=True)
  (qnn4): TorchConnector()
  (dense2): Linear(in_features=2, out_features=1, bias=True)
)


In [330]:
# Training loop
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # Calculate validation loss
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    print(f'Epoch [{epoch + 1}/{num_epochs}] - Train Loss: {train_loss / len(train_loader):.4f}, Val Loss: {val_loss / len(val_loader):.4f}')

print('Training complete!')

IndexError: too many indices for tensor of dimension 2

In [ ]:

# Standardize the input data (optional but recommended)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Convert numpy arrays to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
Y_train = torch.tensor(Y_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
Y_val = torch.tensor(Y_val, dtype=torch.float32)

# Create DataLoader for training and validation
batch_size = 32  # You can adjust this batch size as needed
train_dataset = TensorDataset(X_train, Y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = TensorDataset(X_val, Y_val)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Define training parameters
num_epochs = 50  # Adjust as needed
learning_rate = 0.001  # Adjust as needed

# Define your optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

# Training loop
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # Calculate validation loss
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    print(f'Epoch [{epoch + 1}/{num_epochs}] - Train Loss: {train_loss / len(train_loader):.4f}, Val Loss: {val_loss / len(val_loader):.4f}')

print('Training complete!')


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [ ]:
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=False)

# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)

# X_test = scaler.fit_transform(X_test)

In [ ]:
# X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
# X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

In [ ]:
# X_test = X_test.astype(np.float32)
# Y_train = Y_train.astype(np.float32)
# X_train = X_train.astype(np.float32)
# Y_test = Y_test.astype(np.float32)

In [ ]:
from qiskit import QuantumCircuit

In [ ]:
def create_qnn():
    feature_map = ZZFeatureMap(2)
    ansatz = RealAmplitudes(2, reps=1)
    qc = QuantumCircuit(2)
    qc.compose(feature_map, inplace=True)
    qc.compose(ansatz, inplace=True)

    # REMEMBER TO SET input_gradients=True FOR ENABLING HYBRID GRADIENT BACKPROP
    qnn = EstimatorQNN(
        circuit=qc,
        input_params=feature_map.parameters,
        weight_params=ansatz.parameters,
        input_gradients=True,
    )
    return qnn


qnn4 = create_qnn()

In [ ]:
from qiskit_machine_learning.connectors import TorchConnector
import torch
import torch.nn as nn
from torch import Tensor
from torch.nn import Linear, CrossEntropyLoss, MSELoss
from torch.optim import LBFGS

from qiskit import QuantumCircuit
from qiskit.utils import algorithm_globals
from qiskit.circuit import Parameter
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap
from qiskit_machine_learning.neural_networks import SamplerQNN, EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector

class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.lstm = nn.LSTM(input_size=48, hidden_size=64, batch_first=True, num_layers=1, bidirectional=False)
        self.dropout = nn.Dropout(0.7)
        self.dense1 = nn.Linear(64, 2)
        self.qnn4 = TorchConnector(qnn4)  # Assuming 'qnn4' is your custom layer
        self.dense2 = nn.Linear(2, 1)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.dropout(x)
        x = self.dense1(x[:, -1, :])  # Taking the last output from the LSTM
        x = self.qnn4(x)
        x = self.dense2(x)
        return x

# Instantiate the model
model = CustomModel()

# Print the model architecture
print(model)

# Define your optimizer and loss function
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.MSELoss()

# Print a summary of the model (number of parameters, layer details, etc.)
print(model)

CustomModel(
  (lstm): LSTM(48, 64, batch_first=True)
  (dropout): Dropout(p=0.7, inplace=False)
  (dense1): Linear(in_features=64, out_features=2, bias=True)
  (qnn4): TorchConnector()
  (dense2): Linear(in_features=2, out_features=1, bias=True)
)
CustomModel(
  (lstm): LSTM(48, 64, batch_first=True)
  (dropout): Dropout(p=0.7, inplace=False)
  (dense1): Linear(in_features=64, out_features=2, bias=True)
  (qnn4): TorchConnector()
  (dense2): Linear(in_features=2, out_features=1, bias=True)
)


ValueError: batch_size should be a positive integer value, but got batch_size=      Stacked_Column
0                2.0
1                0.0
2                1.0
3                0.0
4                1.0
...              ...
1985             2.0
1986             1.0
1987             1.0
1988             1.0
1989             0.0

[1990 rows x 1 columns]

In [ ]:
for epoch in range(200):
    total_loss = []
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad(set_to_none=True)  # Initialize gradient
        output = model4(data)  # Forward pass
        loss = loss_func(output, target)  # Calculate loss
        loss.backward()  # Backward pass
        optimizer.step()  # Optimize weights
        total_loss.append(loss.item())  # Store loss

        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        
    loss_list.append(sum(total_loss) / len(total_loss))
    accuracy_list.append(correct / len(train_loader) / batch_size * 100)
    print("Training [{:.0f}%]\tLoss: {:.4f}\tAccuracy {:.2f}%".format(100.0 * (epoch + 1) / epochs, loss_list[-1], correct / len(train_loader) / batch_size * 100))

NameError: name 'train_loader' is not defined

In [ ]:
len(X_train)

1990

In [ ]:
Y_train

,Stacked_Column
0,2.0
1,0.0
2,1.0
3,0.0
4,1.0
...,...
1985,2.0
1986,1.0
1987,1.0
1988,1.0


In [ ]:
history = model.fit(
    X_train, Y_train ,
    epochs=50,  # Adjust the number of epochs as needed
    batch_size=64,  # Adjust the batch size as needed
    validation_data=(X_test, Y_test),
    verbose=1
)

Epoch 1/50
32/32 [==============================] - 0s 2ms/step - loss: 0.4776 - mae: 0.5237 - val_loss: 1.4856 - val_mae: 0.9653
Epoch 2/50
32/32 [==============================] - 0s 2ms/step - loss: 0.4779 - mae: 0.5257 - val_loss: 1.4611 - val_mae: 0.9582
Epoch 3/50
32/32 [==============================] - 0s 2ms/step - loss: 0.4618 - mae: 0.5179 - val_loss: 1.5159 - val_mae: 0.9745
Epoch 4/50
32/32 [==============================] - 0s 2ms/step - loss: 0.4814 - mae: 0.5261 - val_loss: 1.4722 - val_mae: 0.9617
Epoch 5/50
32/32 [==============================] - 0s 2ms/step - loss: 0.4526 - mae: 0.5057 - val_loss: 1.4627 - val_mae: 0.9580
Epoch 6/50
32/32 [==============================] - 0s 2ms/step - loss: 0.4746 - mae: 0.5203 - val_loss: 1.4894 - val_mae: 0.9662
Epoch 7/50
32/32 [==============================] - 0s 2ms/step - loss: 0.4585 - mae: 0.5110 - val_loss: 1.4712 - val_mae: 0.9606
Epoch 8/50
32/32 [==============================] - 0s 2ms/step - loss: 0.4691 - mae: 0.51

In [ ]:
Y_pred = model.predict(X_test)

print(str(Y_pred[1]), " ", Y_test[1])

16/16 [==============================] - 0s 803us/step
[[1.6517398]]   2.0
